In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing
from freezingeffect import selection_of_ROIs, automatic_ROI_propagation, collect_data_propagated
from freezingeffect.helpers import load_parameters_ROIs

## 1. Process the Mueller Matrices of the folders

In [2]:
# set the parameters to run the script
data_folder_path = r'F:\ImpactHistology\Freezing_dev'
directories = [data_folder_path]
calib_directory = r'F:\polarimetry\calibration'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = False, parameter_set = 'PD')

0it [00:00, ?it/s]


[]

## 2. Specify the folders that will be processed and clean them (remove precedent processing results)

In [3]:
path_folder_temp, path_align_folder = selection_of_ROIs.create_alignment_folder()
base_dirs = selection_of_ROIs.get_the_base_dirs(data_folder_path)
selection_of_ROIs.clean_the_alignment_folders(path_align_folder)
selection_of_ROIs.create_the_masks(data_folder_path)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.99it/s]


## 3. Loop over all the folders that will be processed

#### 3.1. loop over all the folders that needs to be processed

In [4]:
# tissue_types = ['WM', 'GM']
tissue_types = ['WM', 'GM']
for path_folder in base_dirs:
    for tissue_type in tissue_types:
        
        # 1. set up the parameters and load the Mueller Matrix
        WM = tissue_type == 'WM'
        
        if WM:
            new_folder_name = 'WM_1'
        else:
            new_folder_name = 'GM_1'
        propagated = False
        
        # 2. automatic selection and preparation for the propagation of the ROIs
        propagation_list = selection_of_ROIs.square_selection(path_folder_temp, path_folder, 
                                                             WM, tissue_type, path_align_folder)
        
        # 3. actually do the alignment
        propagation_list = automatic_ROI_propagation.generate_combined_mask(propagation_list, 
                                                                            path_align_folder)
        automatic_ROI_propagation.do_alignment(path_align_folder)
        output_folders = automatic_ROI_propagation.move_computed_folders(path_align_folder)
        
        # 4. and collect the data after propagation
        collect_data_propagated.collect_data_propagated(WM, path_align_folder, propagation_list, output_folders)

## 4. Compare the parameters

#### 4.1. Get the folders for the measurements of interest

In [14]:
from freezingeffect import parameter_comparaison

In [15]:
# get the parameters for the comparaison
path_folder = data_folder_path
wavelength, match_sequence, measurement, max_ = parameter_comparaison.get_parameters()


# and the folders containing the data
data_folders, base_folders, to_be_removed = parameter_comparaison.get_data_folders(path_folder, wavelength, match_sequence)

data_folders

['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_1',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_3',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_4',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_5',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\WM_1',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\WM_2',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\WM_3',
 'F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\WM_4',
 'F:\\Impa

#### 4.2. Get the data for the measurement

In [32]:
data_types, data_fixation = parameter_comparaison.get_data(data_folders)

data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,100.262060,15.750751,105.586592,100.262060,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.837052,0.020021,0.841176,0.838816,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.038612,0.009783,0.040404,0.038551,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,11.656016,4.555050,10.558659,10.989487,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,2.051196,9.742727,176.983240,2.051196,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.827219,0.034976,0.849265,0.830354,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.024154,0.010026,0.020202,0.023006,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,16.790228,4.315089,16.592179,16.641679,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,161.954085,16.417853,155.865922,161.954085,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.853522,0.021292,0.833088,0.852637,None


In [29]:
# remove the ROI moving from white matter to grey matter and vice versa
for data_type, dfed in data_fixation.items():
    for folder, df in dfed.items():
        to_remove_dfed = to_be_removed[folder.split('\\polarimetry')[0]]
        for val in to_remove_dfed:
            for fol in val:
                if fol[0] == folder.split('\\')[-1].split('_align')[0]:
                    df = df.apply(parameter_comparaison.find_match_seq, match_seq = fol[1], axis = 1)
                    data_fixation[data_type][folder] = df

In [31]:
data_fixation['GM']['F:\\ImpactHistology\\Freezing_dev\\2023-04-06_T_FRE-FR-T0_FR_S1_1\\polarimetry\\550nm\\50x50_images\\GM_2']

,fname,parameter,mean,stdev,max,median,square_size
0,2023-04-06_T_FRE-FR-T0_FR_S1_1,azimuth,100.262060,15.750751,105.586592,100.262060,None
1,2023-04-06_T_FRE-FR-T0_FR_S1_1,depolarization,0.837052,0.020021,0.841176,0.838816,None
2,2023-04-06_T_FRE-FR-T0_FR_S1_1,diattenuation,0.038612,0.009783,0.040404,0.038551,None
3,2023-04-06_T_FRE-FR-T0_FR_S1_1,retardance,11.656016,4.555050,10.558659,10.989487,None
4,2023-04-11_T_FRE-FR-T1_FR_S1_1,azimuth,2.051196,9.742727,176.983240,2.051196,None
5,2023-04-11_T_FRE-FR-T1_FR_S1_1,depolarization,0.827219,0.034976,0.849265,0.830354,None
6,2023-04-11_T_FRE-FR-T1_FR_S1_1,diattenuation,0.024154,0.010026,0.020202,0.023006,None
7,2023-04-11_T_FRE-FR-T1_FR_S1_1,retardance,16.790228,4.315089,16.592179,16.641679,None
8,2023-04-12_T_FRE-FR-T2_FX_S1_1,azimuth,161.954085,16.417853,155.865922,161.954085,None
9,2023-04-12_T_FRE-FR-T2_FX_S1_1,depolarization,0.853522,0.021292,0.833088,0.852637,None


In [35]:
#### Set up the parameters to be studied
parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth']
metrics = ['median', 'mean', 'max']
times = ['T0', 'T1', 'T2']
    
# create the directories that will be used to store the output of the analysis
folder = parameter_comparaison.create_output_directories(measurement, data_types, parameters = parameters)

In [43]:
### Get the data and save it into xlsx file
to_combine = parameter_comparaison.get_and_organize_data(data_fixation, metrics, data_types, times, folder, max_,
                          parameters = ['retardance', 'depolarization', 'diattenuation', 'azimuth'])

to_combine[('retardance', 'WM', 'median')]

,0,1,2,3,4
0,12.743272,11.638080,27.364957,15.164096,12.089109
1,12.946663,11.518749,29.418080,14.294720,13.872351
2,18.935977,17.305498,20.171249,15.184955,21.729510


In [51]:
to_combine_cv = {}
for idx, val in to_combine.items():
    if idx[0] == 'azimuth':
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val, azimuth = True)
    else:
        to_combine_cv[idx] = parameter_comparaison.combine_data_cv(val)
        
to_combine_cv[('retardance', 'WM', 'median')]

,0,1,2,3,4
0,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.015961,0.989747,1.075027,0.942669,1.147508
2,1.485959,1.486972,0.737120,1.001376,1.797445


#### 4.3. Statistical analysis

In [52]:
#### Get the data to be used for the comparisons
# recombine the data in a different format
data_all_recombined = parameter_comparaison.recombine_data_tests(to_combine, times)
data_all_recombined_cv = parameter_comparaison.recombine_data_tests(to_combine_cv, times)

In [56]:
#### Comparaison of the means and the means of the fold change
# perform the statistical test for both fold change and mean values
paired_test = parameter_comparaison.t_test(data_all_recombined, times, 'median')# times[1:]])
paired_test_cv = parameter_comparaison.t_test(data_all_recombined_cv, times, 'median')# times[1:]])
df_grouped = parameter_comparaison.create_test_df(paired_test, parameter = 'median')
df_grouped_cv = parameter_comparaison.create_test_df(paired_test_cv, parameter = 'median')
df_grouped_cv

Z   p_value  n_1  n_2  before/GM  \
parameter      data_type time                                        
azimuth        GM        T0    25.0  0.007495  5.0  5.0  48.455844   
                         T1    12.5  1.000000  5.0  5.0  48.455844   
                         T2    16.0  0.547619  5.0  5.0  48.455844   
               WM        T0    25.0  0.007495  5.0  5.0  33.060046   
                         T1    12.5  1.000000  5.0  5.0  33.060046   
                         T2     8.0  0.420635  5.0  5.0  33.060046   
depolarization GM        T0     5.0  0.118797  5.0  5.0   0.982001   
                         T1    12.5  1.000000  5.0  5.0   0.982001   
                         T2     0.0  0.007937  5.0  5.0   0.982001   
               WM        T0    10.0  0.655836  5.0  5.0   0.997003   
                         T1    12.5  1.000000  5.0  5.0   0.997003   
                         T2    10.0  0.690476  5.0  5.0   0.997003   
retardance     GM        T0    10.0  0.655836  5.0  5.0   0.919043   
                         T1    12.5  1.000000  5.0  5.0   0.919043   
                         T2    17.0  0.420635  5.0  5.0   0.919043   
               WM        T0    15.0  0.655836  5.0  5.0   1.034182   
                         T1    12.5  1.000000  5.0  5.0   1.034182   
                         T2     8.0  0.420635  5.0  5.0   1.034182   

                               before/GM std   after/WM  after/WM std  
parameter      data_type time                                          
azimuth        GM        T0        30.638565   0.000000      0.000000  
                         T1        30.638565  48.455844     30.638565  
                         T2        30.638565  36.356334     23.832364  
               WM        T0         6.462226   0.000000      0.000000  
                         T1         6.462226  33.060046      6.462226  
                         T2         6.462226  36.874311      6.444848  
depolarization GM        T0         0.023541   1.000000      0.000000  
                         T1         0.023541   0.982001      0.023541  
                         T2         0.023541   1.076826      0.047065  
               WM        T0         0.006553   1.000000      0.000000  
                         T1         0.006553   0.997003      0.006553  
                         T2         0.006553   0.998112      0.004819  
retardance     GM        T0         0.390272   1.000000      0.000000  
                         T1         0.390272   0.919043      0.390272  
                         T2         0.390272   0.722533      0.286377  
               WM        T0         0.070982   1.000000      0.000000  
                         T1         0.070982   1.034182      0.070982  
                         T2         0.070982   1.301774      0.380259

In [57]:
# finally, save the results
path_statistical_test = os.path.join('results', measurement)
df_grouped.to_excel(os.path.join(path_statistical_test, 'statistical_test.xlsx'))
df_grouped_cv.to_excel(os.path.join(path_statistical_test, 'statistical_test_cv.xlsx'))

#### 4.4. Combine the results in a single excel file

In [59]:
# this cell allows to combine the data obtained for both grey and white matter in a table that can then be used
# to plot the results using Graphpad Prism
retardance = {}
depolarization = {}
for key, param in to_combine.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization[key[1]] = param
        
retardance_cv = {}
depolarization_cv = {}
azimuth_cv = {}
for key, param in to_combine_cv.items():
    if key[0] == 'retardance' and key[2] == 'median':
        retardance_cv[key[1]] = param
    elif key[0] == 'depolarization' and key[2] == 'median':
        depolarization_cv[key[1]] = param
    else:
        if key[2] == 'mean':
            azimuth_cv[key[1]] = param
        
retardance = parameter_comparaison.create_df_prism(retardance, max_)
depolarization = parameter_comparaison.create_df_prism(depolarization, max_)
    
retardance_cv = parameter_comparaison.create_df_prism(retardance_cv, max_)
depolarization_cv = parameter_comparaison.create_df_prism(depolarization_cv, max_)
azimuth_cv = parameter_comparaison.create_df_prism(azimuth_cv, max_)

retardance_cv

,0,1,2,3,4,5,6,7,8,9
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.672238,1.514327,1.237046,0.693176,0.478428,1.015961,0.989747,1.075027,0.942669,1.147508
2,1.138663,0.975022,0.539669,0.583883,0.375429,1.485959,1.486972,0.737120,1.001376,1.797445


In [60]:
# finally save all the tables
path_data_prism = os.path.join('results', measurement)
retardance.to_excel(os.path.join(path_data_prism, "retardance_prism.xlsx"))
depolarization.to_excel(os.path.join(path_data_prism, "depolarization_prism.xlsx"))
retardance_cv.to_excel(os.path.join(path_data_prism, "retardance_prism_cv.xlsx"))
depolarization_cv.to_excel(os.path.join(path_data_prism, "depolarization_prism_cv.xlsx"))
azimuth_cv.to_excel(os.path.join(path_data_prism, "azimuth_prism_cv.xlsx"))